In [1]:
import pandas as pd
import os
from sklearn import preprocessing
from collections import deque
import random
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [2]:
SEQ_LEN = 60 #last 60 minutes
FUTURE_PERIOD_PREDICT = 3 #future 3 minutes
RATIO_TO_PREDICT = "LTC-USD"
EPOCHS = 10
BATCH_SIZE = 64
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

In [3]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [4]:
def preprocess_df(df):
    df = df.drop("future", 1)
    
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
            
    df.dropna(inplace=True)
    
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])
    
    random.shuffle(sequential_data)
    
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
            
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys+sells   
    random.shuffle(sequential_data)
    
    X = []
    y = []
    
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
        
    return np.array(X), y

In [5]:
main_df = pd.DataFrame()

In [6]:
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    
    df = pd.read_csv(dataset, names=["time","low","high","open","close","volume"])
    print(df.head())

         time          low         high         open        close    volume
0  1528968660  6489.549805  6489.560059  6489.560059  6489.549805  0.587100
1  1528968720  6487.370117  6489.560059  6489.549805  6487.379883  7.706374
2  1528968780  6479.410156  6487.370117  6487.370117  6479.410156  3.088252
3  1528968840  6479.410156  6479.419922  6479.419922  6479.410156  1.404100
4  1528968900  6475.930176  6479.979980  6479.410156  6479.979980  0.753000
         time        low       high       open      close      volume
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067
4  1528968900  96.279999  96.540001  96.500000  96.389999  524.539978
         time        low   high        open      close     volume
0  1528968720  485.98999  486.5  486.019989  486.01001  26

In [7]:
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    
    df = pd.read_csv(dataset, names=["time","low","high","open","close","volume"])
    
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

In [8]:
main_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume
time,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500


In [9]:
main_df["future"] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

In [10]:
main_df[[f"{RATIO_TO_PREDICT}_close", "future"]].head()

,LTC-USD_close,future
time,,
1528968660,96.580002,96.500000
1528968720,96.660004,96.389999
1528968780,96.570000,96.519997
1528968840,96.500000,96.440002
1528968900,96.389999,96.470001


In [11]:
main_df["target"] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

In [12]:
main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head()

,LTC-USD_close,future,target
time,,,
1528968660,96.580002,96.500000,0
1528968720,96.660004,96.389999,0
1528968780,96.570000,96.519997,0
1528968840,96.500000,96.440002,0
1528968900,96.389999,96.470001,1


In [13]:
times = sorted(main_df.index.values)

In [14]:
last_5pct = times[-int(0.05*len(times))]
last_5pct

1534922100

In [15]:
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

In [16]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [17]:
print(f"train data: {len(train_x)} , validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)} , buys: {train_y.count(1)}")
print(f"Validation Dont buy: {validation_y.count(0)} buys: {validation_y.count(0)}")

train data: 69188 , validation: 3062
Dont buys: 34594 , buys: 34594
Validation Dont buy: 1531 buys: 1531


In [18]:
model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(LSTM(128, input_shape=(train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

In [20]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=opt,
             metrics=["accuracy"])
tensorboard = TensorBoard(log_dir=f"logs/{NAME}")

In [21]:
filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  #unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor="val_acc", verbose=1, save_best_only=True, mode="max")) #saves only best ones

In [22]:
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint])

Train on 69188 samples, validate on 3062 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
69188/69188 [==============================] - 544s 8ms/sample - loss: 0.7217 - acc: 0.5155 - val_loss: 0.6875 - val_acc: 0.5330
Epoch 2/10
69188/69188 [==============================] - 547s 8ms/sample - loss: 0.6920 - acc: 0.5232 - val_loss: 0.6950 - val_acc: 0.5020
Epoch 3/10
69188/69188 [==============================] - 540s 8ms/sample - loss: 0.6928 - acc: 0.5143 - val_loss: 0.6861 - val_acc: 0.5601
Epoch 4/10
69188/69188 [==============================] - 574s 8ms/sample - loss: 0.6879 - acc: 0.5436 - val_loss: 0.6832 - val_acc: 0.5516
Epoch 5/10
69188/69188 [==============================] - 519s 8ms/sample - loss: 0.6848 - acc: 0.5581 - val_loss: 0.6800 - val_acc: 0.5601
Epoch 6/10
69188/69188 [==============================] - 504s 7ms/sample - loss: 0.6826 - acc: 0.5625 - val_loss: 0.6935 - val_acc: 0.5160
Epoch 7/10
69188/69188 [==============================] - 474s 